<a href="https://colab.research.google.com/github/alexweltgeist/alexweltgeist/blob/main/projekt_automarken.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Automarken erkennen**

### Ziel

Ziel des Projektes ist es mittels eines CNN auf Fotos die Automarke des dargestellten Vehikels zu erkennen. Falls wir genügend Zeit haben, werden wir auch versuchen aufgrund des Bildes auf die Preisklasse des Autos zu schliessen.


### Datenbasis

Als Datenbasis sind verschiedene Bildsammlungen zugänglich wobei der Datensatz der Stanford University von Jonathan Krause der wohl bekannteste ist (https://ai.stanford.edu/~jkrause/cars/car_dataset.html). Allerdings ist dieser Datensatz nicht besonders gross, etwas USA lastig und schon etwas in die Jahre gekommen. 

Wir verwenden daher eine neue Library von Nicolas Gervais (https://github.com/nicolas-gervais/predicting-car-price-from-scraped-data/tree/master/picture-scraper) mit 64'467 Bilder aus dem Jahr 2020. Die Bilder wurden von der Webpage 'The Car Connection' gescraped und haben für jedes BIld unter anderem die folgenden Label: 

*   Marke	(z.B. Audi)
*   Modell	(z.B. A5)
*   Baujahr	(z.B. 2013)
*   Preis in '000 USD  (z.B. 44)

Auf den Bildern sind zum Teil auch Detailaufnahmen einzelner Autoteile zu sehen und vereinzelt sind auch Innenaufnahmen vorhanden. Die Bilder haben unterschiedliche Formate und Grössen.





### Vorbereitung 

Zunächst werden die benötigten Libaries geladen.

In [1]:
import itertools, os
from google.colab import files
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
! pip install -q kaggle

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

TF version: 2.4.1
Hub version: 0.11.0
GPU is available


**Daten laden ins Colab**

Versuch die Daten direkt aus Kaggle via API in Colab zu laden: dazu muss ein API-key in Kaggle generiert und lokal gespeichert werden. DIeser muss dann ins Colab directory hochgeladen werden in ein Verzeichnis ~/.kaggle. 
Zudem muss die library 'kaggle' installiert sein.

In [2]:
# Hochladen des Files "kaggle.json" mit dem API key von lokalem drive
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"alexanderfurrer","key":"b77d9ebcec2f43617c7fefc1d4e1c736"}'}

In [3]:
# File ins richtige Verzeichnis legen
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! dir

kaggle.json  sample_data


In [ ]:
# Code um allenfalls bestehende Verzeichnisse oder FIles zu löschen
# ! rm ~/.kaggle/kaggle.json
# ! rm ~/kaggle.json
# ! rmdir ~/.kaggle

In [4]:
# Testen des Zugriffs auf Kaggle mit einer Anzeiger von neuen Datasets
! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              217KB  2021-03-24 09:00:10            882  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-02-17 16:10:44            697  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   1GB  2021-02-18 10:08:27            657  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19            276  
fatiimaezzahra/famous-iconic-women                          Famous Iconic Wo

In [5]:
# Laden der Daten
! kaggle datasets download -d prondeau/the-car-connection-picture-dataset

100% 681M/682M [00:25<00:00, 28.6MB/s]
100% 682M/682M [00:25<00:00, 27.5MB/s]


In [ ]:
# Entpacken ins Verzeichnis Data
! mkdir Data
! unzip the-car-connection-picture-dataset.zip -d Data

### 1. Bildmaterial aufbereiten

*   Kategorien aus den Bildnamen extrahieren
*   Subfolder mit Kategorien erstellen
*   Bilder zurechtschneiden (muss das quadratisch sein?)
*   Grösse skalieren (was ist vernünftig? 200 * 250?)
*   Bilder geeignet abspeichern (HDF5 format nötig?)



In [ ]:
# Subfolder erstellen

import os, shutil, glob

#Source file 
sourcefile = 'C:/Users/alex/CAS_ML_local/B_Deeplearning/03_Project/Data2'

# read the file names in the source file directory
for filename in os.listdir(sourcefile): 
    brand = filename.rsplit('_', 17)[0]
    # If folder does not exist try making new one
    try:
        os.mkdir(os.path.join(sourcefile, brand))
    # except error then pass
    except WindowsError:
        pass
    # Move the images from file to new folder based on image name
    shutil.move(os.path.join(sourcefile, filename), os.path.join(sourcefile, brand, filename))


### 2. Datensatz vorbereiten

*   Split in Training (55'000), Validierung (5'000) und Test (4'467)

### 3. Modell definieren

*   Vortrainiert oder 'from scratch'?
*   Anzahl layer? 
*   Aktivierungsfunktion?